<a href="https://colab.research.google.com/github/Kkiirraan/RAG/blob/main/ResumeReader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community langchain-google-genai pypdf

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install langchain-chroma

In [ ]:
!pip install langchain-google-genai

**All the imports**

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

**1. Load a single PDF from a local path**

In [ ]:
loader = PyPDFLoader("/content/RESUME_KiranVijayan__.pdf")
documents = loader.load()

**2. Initialize text splitter**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

In [ ]:
chunks = text_splitter.split_documents(documents)

In [ ]:
print(f"Original document has {len(documents[0].page_content)} characters.")
print(f"Split into {len(chunks)} chunks.")

Original document has 3728 characters.
Split into 5 chunks.


**3. Embedding**

In [ ]:
model_name = "BAAI/bge-small-en-v1.5"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipython-input-3228418543.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

In [ ]:
chunk_texts = [chunk.page_content for chunk in chunks]
chunk_embeddings = embeddings_model.embed_documents(chunk_texts)

In [ ]:
print(f"Number of chunks: {len(chunk_embeddings)}")
print(f"Embedding dimension of a single chunk: {len(chunk_embeddings[0])}")
print("Example of an embedding vector (first 5 numbers):")
print(chunk_embeddings[0][:5])

Number of chunks: 5
Embedding dimension of a single chunk: 384
Example of an embedding vector (first 5 numbers):
[-0.024127831682562828, -0.010547494515776634, -0.006530714686959982, -0.0090232715010643, -0.01712646149098873]


**4. Vector Store**

In [ ]:
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model,
    persist_directory="./chroma_db"
)

In [ ]:
retriever = vector_store.as_retriever()

**5. Augmented Prompt**

In [ ]:
prompt_template = """
You are a resume chatbot 🤖. Your purpose is to answer a user's question directly and concisely, using **only** the information provided in the resume text.

- If the answer is found, provide a short and direct response.
- If the answer is not present in the resume, you must respond with: "I'm sorry, that information is not available in the provided resume. 🧐"

**Resume Text:**
{context}

**Question:** {question}
**Answer:**
"""

**6. Generation**

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [ ]:
query = "Name of his college"

In [ ]:
retrieved_documents = retriever.invoke(query)

In [ ]:
context = ""
for doc in retrieved_documents:
    context += doc.page_content + "\n\n"

In [ ]:
augmented_prompt = prompt_template.format(context=context, question=query)

In [ ]:
final_answer = llm.invoke(augmented_prompt)

In [ ]:
final_answer.content

'College of Engineering Kidangoor'